# Essa instância do colab será responsável por verificar e explicar os insights observados nos dados do cliente.

In [ ]:
from google.colab import drive #Importanto o drive do email que você utilizou para se conectar ao colab
drive.mount('/content/drive')  #Estou conectando o meu drive ao colab, já que vou puxar o csv utilizado na resolução do desafio de lá.
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import seaborn as sns
import plotly.express as px
from tensorflow import keras
from tensorflow.keras import layers
pd.options.display.float_format = '{:,.0f}'.format  #Para a visualização melhor dos dados de faturamento (não ficar, por exemplo, "e+....")


#O caminho passado aqui deve ser diferente dependendo de como esteja salvo este arquivo no seu drive. Ou seja, é so passar na função "read_csv" o caminho
#do arquivo em csv realizado na resolução do desafio do seu drive. 
desafio = pd.read_csv('/content/drive/My Drive/DesafioEstagio/DesafioEstagioMachineLearning.csv')

Mounted at /content/drive


In [ ]:
# Verificando as correlações entre os dados
corr = desafio.corr()
corr.style.background_gradient(cmap='coolwarm')

,codigo,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
codigo,1.000000,0.058340,0.116533,0.112639,0.106236,0.088400,0.083265,0.075811,0.014112,-0.089280,-0.161552,-0.212292,-0.220039,-0.066785,0.072148,0.158117,0.187358,0.186055,-0.273532,-0.381192
população,0.058340,1.000000,0.970797,0.972057,0.985313,0.990373,0.989377,0.986526,0.991235,0.924673,0.186290,0.293839,0.565726,0.905305,0.928378,0.861343,0.832363,0.822980,-0.026747,-0.166563
popAte9,0.116533,0.970797,1.000000,0.999140,0.994495,0.988292,0.949930,0.934287,0.946176,0.822485,0.043099,0.118759,0.384971,0.790162,0.870195,0.831198,0.808531,0.781555,-0.071316,-0.300307
popDe10a14,0.112639,0.972057,0.999140,1.000000,0.995850,0.987774,0.948990,0.936012,0.949140,0.826978,0.029706,0.108616,0.383258,0.797436,0.876245,0.835387,0.813732,0.784861,-0.078434,-0.306282
popDe15a19,0.106236,0.985313,0.994495,0.995850,1.000000,0.995039,0.970848,0.961319,0.962284,0.852647,0.059684,0.147259,0.425413,0.835775,0.910146,0.869813,0.846498,0.821901,-0.075903,-0.293721
popDe20a24,0.088400,0.990373,0.988292,0.987774,0.995039,1.000000,0.981342,0.967448,0.969580,0.873030,0.103014,0.200898,0.474706,0.856208,0.913433,0.861188,0.833742,0.809816,-0.059987,-0.250416
popDe25a34,0.083265,0.989377,0.949930,0.948990,0.970848,0.981342,1.000000,0.994647,0.969477,0.902734,0.198600,0.303916,0.563934,0.907814,0.951993,0.902184,0.874031,0.863436,-0.037166,-0.195332
popDe35a49,0.075811,0.986526,0.934287,0.936012,0.961319,0.967448,0.994647,1.000000,0.973315,0.916271,0.224111,0.328107,0.588285,0.924404,0.958461,0.905541,0.878573,0.873529,-0.025153,-0.170591
popDe50a59,0.014112,0.991235,0.946176,0.949140,0.962284,0.969580,0.969477,0.973315,1.000000,0.956639,0.247188,0.363377,0.634230,0.924839,0.902424,0.807894,0.774645,0.773025,0.009856,-0.067345
popMaisDe60,-0.089280,0.924673,0.822485,0.826978,0.852647,0.873030,0.902734,0.916271,0.956639,1.000000,0.397839,0.550859,0.802572,0.956781,0.846138,0.715222,0.677137,0.703864,0.074336,0.139008


**Verificando a correlação entre os dados apresentados: Azul escuro para uma correlação menor entre os valores até laranja escuro, para uma correlação maior entre os valores.**

Verifica-se que os valores de faturamente estão altamente correlacionados aos valores de Domicilios de A1, A2 e B1, respectivamente, o que nos faz supor que procurar bairros aonde esses três tipos de domicilios sejam mais presentes seja uma boa ideia. 

Algo semelhante pode-se dizer dos domicilios C2 e D, já que eles são os com menores correlação com os valores de faturamento. Faz-se por entender, então, que talvez não seja uma boa ideia investir em bairros na qual a maior parte de seus domicílios estão presentes nesses dois tipos. 

Outro fator que podemos notar é que, tratando-se da população, a que tem uma maior correlação com os valores de Faturamento é a população com mais de 60 anos, que é também a população que mais tem relação com os domicílios de A1, A2 e B1, que são nossos domicílios que mais influenciam no faturamento, o que comprova nossa teoria.



In [ ]:
Domicilios = desafio.drop(["codigo", "população",	"popAte9", "popDe10a14", "popDe15a19",	"popDe20a24",	"popDe25a34",	"popDe35a49",	
                                   "popDe50a59",	"popMaisDe60", "rendaMedia",	"faturamento", "nome", "cidade", "estado"], axis=1)
Domicilios

,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE
0,0,145,715,1242,1093,758,92,304
1,0,0,82,506,2040,2490,827,2506
2,0,96,404,652,644,522,77,254
3,114,178,393,517,945,584,137,286
4,0,0,1089,2821,5110,5422,1073,3261
...,...,...,...,...,...,...,...,...
155,0,259,774,1363,1923,1379,190,405
156,0,219,714,828,1051,1012,116,465
157,0,906,2866,2241,2982,1200,360,772
158,0,87,556,836,940,484,75,214


In [ ]:
Domicilios.sum()

domiciliosA1     62156
domiciliosA2     97323
domiciliosB1    299450
domiciliosB2    381612
domiciliosC1    566376
domiciliosC2    484929
domiciliosD      90940
domiciliosE     291458
dtype: int64

In [ ]:
bar = Domicilios.sum()
fig = px.bar(x=bar.index , y=bar.values , text=bar.values , labels={'x':'Tipo do domicilio','y':'Número'})
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside',
                 marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                 marker_line_width=1.5, opacity=.7)

fig.update_layout(title_text='Quantidade de domicilios')
fig.show()

  Como pode-se ver nos gráficos, os domicilios do tipo **A1** e **A2** são dois dos menos presentes nos bairros, o que talvez seja um indicativo de que utiliza-los como principal paramêtro para medir o faturamento talvez não seja tão legal, já que será dificil encontrar bairros no qual esses dois tipos de domicilios sejam bastante presentes. 

  O que se pode fazer é, na verdade, procurar bairros que contenham uma grande existência de domicilios do tipo **B1** e **B2**, já que eles são consideravelmente mais presentes nos bairros e ainda assim tem uma boa correlação com os valores de faturamento.
  

In [ ]:
População = desafio.drop(["codigo", "domiciliosA1",	"domiciliosA2",	"domiciliosB1",	"domiciliosB2",	"domiciliosC1",	"domiciliosC2",	"domiciliosD",	
                          "domiciliosE" , "rendaMedia",	"faturamento", "nome", "cidade", "estado"], axis=1)
População

,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60
0,11676,1027,483,688,800,1675,2300,1784,2919
1,27564,5131,2188,2697,2630,4810,5308,2403,2397
2,9003,883,399,597,762,1755,2076,1112,1419
3,9606,1072,538,660,685,1317,2007,1341,1986
4,57222,7677,3774,4892,4600,8660,12272,7157,8190
...,...,...,...,...,...,...,...,...,...
155,18791,1802,921,1442,1462,2500,3944,2920,3800
156,13556,1888,787,1042,982,2542,2908,1528,1879
157,33190,3166,1619,2255,2326,4752,7183,4950,6939
158,8867,724,387,580,565,1205,1812,1474,2120


In [ ]:
População.sum()

população      6824696
popAte9         852650
popDe10a14      397511
popDe15a19      523545
popDe20a24      535901
popDe25a34     1053516
popDe35a49     1412298
popDe50a59      853162
popMaisDe60    1196113
dtype: int64

In [ ]:
PopIdade = População.drop('população', axis=1)
PopIdade

,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60
0,1027,483,688,800,1675,2300,1784,2919
1,5131,2188,2697,2630,4810,5308,2403,2397
2,883,399,597,762,1755,2076,1112,1419
3,1072,538,660,685,1317,2007,1341,1986
4,7677,3774,4892,4600,8660,12272,7157,8190
...,...,...,...,...,...,...,...,...
155,1802,921,1442,1462,2500,3944,2920,3800
156,1888,787,1042,982,2542,2908,1528,1879
157,3166,1619,2255,2326,4752,7183,4950,6939
158,724,387,580,565,1205,1812,1474,2120


In [ ]:
bar2 = PopIdade.sum()
fig = px.bar(x=bar2.index , y=bar2.values , text=bar2.values , labels={'x':'Idade','y':'Número'})
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside',
                 marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                 marker_line_width=1.5, opacity=.7)

fig.update_layout(title_text='Idade da população')
fig.show()

Como podemos observar no gráfico, grande parte do público está contida nas idades entre **25 a 49 anos**, com uma grande presença também no público de **mais de 60 anos**. Se considerarmos como público ativo apenas os grupos de **20 a 24 anos** em diante, e considerando as observações de correlações vistas anteriormente de que a população de **35 a 49 anos** está altamente correlacionada com domicílios do tipo B1 em diante, a distribuição da idade atualmente é um bom indicativo de faturamento para a empresa.